In [3]:
import numpy as np

In [4]:
def calculate_profit(decision,P_init,n0):
    #decision=[x1,x2,x3,y,z,gamma,eta]
    #半成品1和半成品2的工续完全一样，考虑为一种情况，减少决策变量
    #储存决策变量(xi:是否检测第i件零件;yi:是否检测第i件半成品;zi:是否拆解第i件不合格的半成品;gamma:是否检测成品;eta:是否拆解不合格成品)
    C=[1,1,2]#储存各零件的检测成本
    P_semi_product=0.1
    P_product=0.1
    C_assemble_semi_product=8
    C_test_semi_product=4
    C_disassemble_semi_product=6
    C_assemble_product=8
    C_test_product=6
    C_disassemble_product=10
    C_market=200
    C_lose=40
    #阶段1:检测零件
    P_upgrade=[(1-decision[i])*P_init[i] for i in range(len(P_init))]
    C_test_part_total=sum([decision[i]*n0*C[i] for i in range(3)])
    #阶段2:装配半成品
    n_semi_product1=min((1-decision[0]*P_init[0])*n0,(1-decision[1]*P_init[1])*n0,(1-decision[2]*P_init[2])*n0)
    C_assemble_semi_product_total=C_assemble_semi_product*n_semi_product1
    #阶段3:是否检测半成品
    C_test_semi_product_total=decision[3]*n_semi_product1*C_test_semi_product
    #未检测前半成品的总次品率
    P_semi_product1_total=(1-P_upgrade[0])*(1-P_upgrade[1])*(1-P_upgrade[2])*P_semi_product+P_upgrade[0]+P_upgrade[1]+P_upgrade[2]-P_upgrade[0]*P_upgrade[1]-P_upgrade[1]*P_upgrade[2]-P_upgrade[0]*P_upgrade[2]+P_upgrade[0]*P_upgrade[1]*P_upgrade[2]
    #更新半成品总次品率，以便计算成品的总次品率
    P_semi_product1_total_upgrade=(1-decision[3])*P_semi_product1_total
    #阶段4:是否拆解不合格半成品
    C_disassemble_semi_product_total=(decision[4]*n_semi_product1*P_semi_product1_total)*C_disassemble_semi_product
    #阶段5:装配成品
    #可装配的成品数量
    n=np.floor(1-decision[3]*P_semi_product1_total)*n_semi_product1
    C_assemble_product_total=n*C_assemble_product
    #阶段6:是否检测成品
    P_product_total=(1-P_semi_product1_total_upgrade)*(1-P_semi_product1_total_upgrade)*(1-P_semi_product1_total_upgrade)*P_product+P_semi_product1_total_upgrade+P_semi_product1_total_upgrade+P_semi_product1_total_upgrade-P_semi_product1_total_upgrade*P_semi_product1_total_upgrade-P_semi_product1_total_upgrade*P_semi_product1_total_upgrade-P_semi_product1_total_upgrade*P_semi_product1_total_upgrade+P_semi_product1_total_upgrade*P_semi_product1_total_upgrade*P_semi_product1_total_upgrade
    C_test_product_total=decision[5]*n*C_test_product
    #阶段7:销售
    Sale=n*(1-P_product_total)*C_market
    C_lose_total=(1-decision[5])*C_lose*n*P_product_total
    #阶段8:是否拆解不合格成品
    C_disassemble_product_total=decision[6]*P_product_total*n*C_disassemble_product
    #最终的目标函数
    profit=Sale-C_lose_total-(C_test_part_total-C_assemble_semi_product_total-C_disassemble_semi_product_total-C_assemble_product_total-C_test_product_total-C_disassemble_product_total)*3+C[0]*n0*decision[0]
    n_upgrade=np.floor(decision[4]*n_semi_product1*P_semi_product1_total)
    return profit,P_upgrade,n_upgrade

In [5]:
decision=[]
for x1 in [0,1]:
    for x2 in [0,1]:
        for x3 in [0,1]:
            for y in [0,1]:
                for z in [0,1]:
                    for gamma in [0,1]:
                        for eta in [0,1]:
                            decision.append([x1,x2,x3,y,z,gamma,eta])

In [6]:
result=[]
for i in decision:
    P_init=[0.1 for i in range(3)]
    n0=10000
    profit,P_upgrade,n_upgrade=calculate_profit(i,P_init,n0)
    result.append([i,profit,P_upgrade,n_upgrade])

In [7]:
non_zero_index=[]
for i in range(2**7):
    if (result[i][3]!=0)&(result[i][1]>0):
        non_zero_index.append(i)

In [8]:
result1=np.empty((2**7,2**7), dtype=object)
for i in non_zero_index:
    for j in range(2**7):
        profit,P_upgrade,n_upgrade=calculate_profit(decision[j],result[i][2],result[i][3])
        result1[i,j]=[decision[j],profit,P_upgrade,n_upgrade]

In [9]:
non_zero_index1=[]
for i in non_zero_index:
    for j in range(2**7):
        if (result1[i][j][3]!=0)&(result1[i][j][1]>0):
            non_zero_index1.append([i,j])

In [10]:
result2=np.empty((2**7,2**7,2**7), dtype=object)
for i in non_zero_index1:
    for j in range(2**7):
        profit,P_upgrade,n_upgrade=calculate_profit(decision[j],result1[i[0]][i[1]][2],result1[i[0]][i[1]][3])
        result2[i[0],i[1],j]=[decision[j],profit,P_upgrade,n_upgrade]

In [11]:
result2_value=np.zeros((2**7,2**7,2**7))
for i in non_zero_index1:
    for j in range(2**7):
        result2_value[i[0]][i[0]][j]=result2[i[0]][i[0]][j][1]
result2_value_max=np.max(result2_value,axis=2)

In [12]:
result1_value=np.zeros((2**7,2**7))
for i in non_zero_index:
    for j in range(2**7):
        result1_value[i,j]=result1[i,j][1]
        result1_value[i,j]=result1[i,j][1]+result2_value_max[i,j]
result1_value_max=np.max(result1_value,axis=1)

In [13]:
result_value=np.zeros(2**7)
for i in range(2**7):
    result_value[i]=result[i][1]
    result_value[i]=result_value[i]+result1_value_max[i]
max_index = np.argmax(result_value)
max_value = result_value[max_index]
print(max_index)
print(max_value)
print("第一阶段的决策为:")
print(result[max_index])

7
2163747.091137528
第一阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 1454019.1908159298, [0.1, 0.1, 0.1], 3439.0]


In [14]:
max_index1= np.argmax(result1_value[max_index])
print(max_index1)
print("第二阶段的决策为:")
print(result1[max_index][max_index1])

7
第二阶段的决策为:
[[0, 0, 0, 0, 1, 1, 1], 500037.19972159824, [0.1, 0.1, 0.1], 1182.0]


In [15]:
max_index2= np.argmax(result2_value[max_index][max_index1])
print(max_index2)
print("第三阶段的决策为:")
print(result2[max_index][max_index1][max_index2])

119
第三阶段的决策为:
[[1, 1, 1, 0, 1, 1, 1], 209690.70059999998, [0.0, 0.0, 0.0], 106.0]
